<a href="https://colab.research.google.com/github/ifalfahri/indobertweet-sentiment-analysis-kurikulum-merdeka/blob/main/text_cleaner_indobertweet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 15.9 MB/s eta 0:00:00


In [7]:
import pandas as pd
import re
import emoji
import html

In [4]:
# Fungsi untuk menemukan URL dalam string
def find_url(string):
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    url = re.findall(regex,string)
    return [x[0] for x in url]

In [5]:
# Preprocessing tweet
def preprocess_tweet(tweet):
    # Menghilangkan karakter HTML yang terenkode (contohnya &amp; menjadi &)
    tweet = html.unescape(tweet)

    # Mengubah semua teks menjadi huruf kecil dan mengonversi emoticon ke teks
    tweet = emoji.demojize(tweet).lower()

    # Mengganti mention dan URL dengan placeholder
    new_tweet = []
    for word in tweet.split():
        # Jika kata dimulai dengan @ (mention)
        if word[0] == '@' or word == '[username]':
            new_tweet.append('@USER')
        # Jika kata tersebut adalah URL
        elif find_url(word) != []:
            new_tweet.append('HTTPURL')
        # Ganti string 'httpurl' atau '[url]' dengan 'HTTPURL'
        elif word == 'httpurl' or word == '[url]':
            new_tweet.append('HTTPURL')
        else:
            # Kata selain mention dan URL tetap dipertahankan
            new_tweet.append(word)

    return ' '.join(new_tweet)

In [14]:
# Membaca dataset CSV
df = pd.read_csv('kurikulum-merdeka.csv')  # Gantilah dengan path file CSV Anda

In [15]:
# Memilih kolom teks saja dengan tanggal, atur sesuai selera
df = df[['full_text', 'created_at']]

In [16]:
# Melakukan preprocessing pada kolom 'full_text'
df['processed_text'] = df['full_text'].apply(preprocess_tweet)

In [17]:
# Hapus duplikasi

initial_count = df.shape[0]  # Jumlah df awal
df = df.drop_duplicates(subset=['processed_text'], keep='first')
final_count = df.shape[0]  # Jumlah data setelah duplikasi dihapus

# Menampilkan jumlah data yang dihapus
print(f"Jumlah data sebelum penghapusan duplikat: {initial_count}")
print(f"Jumlah data setelah penghapusan duplikat: {final_count}")
print(f"Jumlah data yang dihapus: {initial_count - final_count}")

Jumlah data sebelum penghapusan duplikat: 5964
Jumlah data setelah penghapusan duplikat: 5752
Jumlah data yang dihapus: 212


In [18]:
# Menyimpan hasil ke file CSV baru
df.to_csv('preprocessed-kurikulum-merdeka.csv', index=False)

In [19]:
# Cek hasil
df.head()

,full_text,created_at,processed_text
0,Pemkot Targetkan Kurikulum Merdeka Terealisasi...,Fri Dec 30 23:50:55 +0000 2022,pemkot targetkan kurikulum merdeka terealisasi...
1,@rudyjatmiko @AlphaARachman @hnurwahid @Syaifu...,Fri Dec 30 22:45:09 +0000 2022,@USER @USER @USER @USER @USER @USER @USER @USE...
2,@motodobel @AlphaARachman @SyaifulHooda @DPR_R...,Fri Dec 30 22:41:47 +0000 2022,@USER @USER @USER @USER @USER @USER @USER @USE...
3,anak kelas 10 yang kurikulum merdeka itu nant...,Fri Dec 30 22:20:51 +0000 2022,anak kelas 10 yang kurikulum merdeka itu nanti...
4,Kepala Badan Standar Kurikulum dan Asesmen Pen...,Fri Dec 30 18:45:30 +0000 2022,kepala badan standar kurikulum dan asesmen pen...
